### Phenotype Predicting with SNP
- In this code, target phenotype is NIH_totcomp
- If you want to see a binary output(or classification), you have to modify "if not use_rf:" part

#### [Info]
- epoch:100
- m = 64
- classify_middle = [16]
- all_layers = [768, 512, 256]
- plotted MSE loss decrement
- loss and r_squared score are saved in loss_result

#### 잘 돌아가면 combine 1:4로 나눠서 train/test 따로 해보기

In [7]:
# =============== input files ===============
import pandas as pd
import numpy as np
combine = pd.read_csv('/home/ubuntu/gps_and_phe_nihtotcomp_notnull.csv', header=0)
combine = combine.set_index('KEY')

In [8]:
loss_result = pd.DataFrame(columns=['epoch', 'reconstuct loss', 'regression loss', 'r squared'])

In [9]:
# =============== constants ===============
k_fold = 10
num_samples = len(combine)
in_channel = 524503-6 
each_fold_size = int(np.ceil(num_samples/k_fold))

# the following are used to normalize data
max_abcd = 3.0
max_label = 5.0
min_label = -5.0


In [10]:
# =============== parameters ===============
#all_layers = [10000, 5000, 1000] # layers in ae, can also be changed to a command line parameter
m = 64
classify_middle = [16]
all_layers = [768, 512, 256]

num_epochs = 100
batch = 50
lr = 0.01
debug = False
binary_output = False
use_rf = False

In [11]:
import os
import random
import torch
import torch.nn.functional as F
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from sklearn.model_selection import KFold
import h5py
import gc
from itertools import islice
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
import itertools
import argparse
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt

In [12]:
def seed_everything(seed=1234):
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything()

In [13]:
# =============== define network ===============
class nn1(nn.Module):
    def __init__(self, orig_middles, classify_middles, m, in_channel, y_dim):
        super(nn1, self).__init__()
        self.in_channel = in_channel


        layers = []
        middles = [in_channel] + orig_middles
        self.f0 = nn.Linear(int(in_channel/2), middles[1]).cuda(0)
        self.f1 = nn.Linear(in_channel - int(in_channel/2), middles[1]).cuda(1)
        self.bn = nn.BatchNorm1d(num_features=middles[1]).cuda()
        for i in range(1, len(middles)-1):
            layers.append(nn.Linear(middles[i], middles[i+1]))
            layers.append(nn.BatchNorm1d(num_features=middles[i+1]))
            layers.append(nn.ReLU(True))
        layers.append(nn.Linear(middles[-1], m))
        layers.append(nn.BatchNorm1d(num_features=m))
        self.encoder = nn.Sequential(*layers).cuda()

        layers2 = []
        middles = [m] + orig_middles[::-1]
        for i in range(len(middles)-1):
            layers2.append(nn.Linear(middles[i], middles[i+1]))
            layers2.append(nn.BatchNorm1d(num_features=middles[i+1]))
            layers2.append(nn.ReLU(True))
        self.decoder = nn.Sequential(*layers2).cuda()
        self.e0 = nn.Linear(orig_middles[0], int(in_channel/2)).cuda(0)
        self.e1 = nn.Linear(orig_middles[0], in_channel - int(in_channel/2)).cuda(1)
        
        layers3 = [] # regression/classification
        middles2 = [m] + classify_middles
        for i in range(len(middles2)-1):
            layers3.append(nn.Linear(middles2[i], middles2[i+1]))
            layers3.append(nn.BatchNorm1d(num_features=middles2[i+1]))
            layers3.append(nn.ReLU(True))

        if binary_output:
            layers3.append(nn.Linear(middles2[-1], 2)) # binary classification
            layers3.append(nn.BatchNorm1d(num_features=2))
        else:
            layers3.append(nn.Linear(middles2[-1], y_dim))
            layers3.append(nn.BatchNorm1d(num_features=y_dim))
        self.regression = nn.Sequential(*layers3).cuda()

    def encode_and_predict(self, x):
        new_x = self.f0(Variable(x[:, :int(self.in_channel/2)]).cuda(0))
        new_x += self.f1(Variable(x[:, int(self.in_channel/2):]).cuda(1)).to('cuda:0')
        #new_x = nn.Dropout(p=0.95)(new_x)
        #new_x = self.bn(new_x)
        new_x = nn.ReLU(True)(new_x)

        encode_x = self.encoder(new_x)
        predict_y = self.regression(encode_x)
        return predict_y

    def forward(self, x):
        new_x = self.f0(Variable(x[:, :int(self.in_channel/2)]).cuda(0))
        new_x += self.f1(Variable(x[:, int(self.in_channel/2):]).cuda(1)).to('cuda:0')
        new_x = nn.Dropout(p=0.95)(new_x)
        new_x = self.bn(new_x)
        new_x = nn.ReLU(True)(new_x)
        
        encode_x = self.encoder(new_x)
        labels = self.regression(encode_x)
        decode_x = self.decoder(encode_x)

        return (nn.Sigmoid()(self.e0(decode_x)), nn.Sigmoid()(self.e1(decode_x.cuda(1)))), labels
        #gpu(0)에서 계산한것 -> sigmoid ,  gpu(1)에서 계산한 것 -> sigmoid 이 두개를 순서쌍으로 반환


In [14]:
def train_nn(learning_rate, m, batch, train_folds):
    print('-'*50)
    print('Training........')
    
    model = nn1(all_layers, classify_middle, m, in_channel, y_dim)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)

    for epoch in range(num_epochs):
        print("epoch: ", epoch)
        batch_index = 0
        count=1
        with open('/home/ubuntu/ABCD_genotype/ABCD_QCed_EA_pruned_v2.ped.temp4') as f:
            while True:
                chunk = list(islice(f, batch))

                #print('chunk length: ', len(chunk))
                if not chunk:
                    break
                batch_index += 1
                if debug and batch_index%100 == 0:
                    print('batch:', batch_index)

                if batch_index in train_folds:
                    all_data = []
                    all_label = []
                    for line in chunk:
                        line_sp = line.split('\t')
                        orig_key = "".join(line_sp[0].split('_')[1:])
                        if orig_key in combine.index:
                            all_label.append(Y.loc[orig_key, y_columns[0]])
                            all_data.append([int(i) for i in line_sp[6:] if i])
                    if not all_label:
                        del all_data
                        gc.collect()
                        print('Not found data in this batch!')
                        continue

                    img = torch.FloatTensor(all_data)
                    img = img.view(img.size(0), -1)
                    img = img/max_abcd
                    label = torch.FloatTensor(all_label)
                    label = (label - min_label) / (max_label - min_label) #여기서 정규화한 것 때매 나중에 바꿔줘야함
                    del all_data, all_label
                    gc.collect()
                    
                    # =================== nn1 forward=====================)
                    output, predict_y= model(img)
                    predict_y = predict_y.view(predict_y.size(0))

                    if binary_output:
                        regression_loss = criterion(predict_y, label.cuda().type(torch.long))
                    else:
                        regression_loss = criterion(predict_y, label.cuda().type(torch.float))
                        with torch.no_grad():
                            r_squared = r2_score(label.numpy(), predict_y.cpu().numpy())

                    loss = reconstruct_criterion(output[0], img[:, :int(in_channel/2)].cuda(0))
                    loss += reconstruct_criterion(output[1], img[:, int(in_channel/2):].cuda(1)).cuda(0)
                    
                    '''
                    print('-'*5)
                    
                    print('reconstruct loss:{:.7f}'.format(loss.item()))
                    print('regression loss:{:.7f}'.format(regression_loss.item()))
                    print('R2 score:{:.7f}'.format(r_squared))
                    '''
                    print('count: ', count)
                    #print(loss_result)
                    
                    count += 1

                    # =================== backward ====================
                    optimizer.zero_grad()
                    total_loss = loss + regression_loss
                    total_loss.backward()
                    optimizer.step()
                    del img
                    gc.collect()

                    #if debug:
                    #    return model # early return to save time

        # =================== log ========================
        if (epoch + 1) % 1 == 0:
            print('epoch [{}/{}]'.format(epoch+1,num_epochs))
            print('reconstruct loss:{:.7f}'.format(loss.item()))
            print('regression loss:{:.7f}'.format(regression_loss.item()))
            print('R2 score:{:.7f}'.format(r_squared))
            print('-'*10)
            loss_result.loc[int(epoch)]=[epoch, loss.item(), regression_loss.item(), r_squared]

    return model

In [15]:
model = train_nn(lr, m, batch, train_folds)
model

NameError: name 'train_folds' is not defined

In [9]:
def make_prediction(test_fold, model, batch, mode=0):
    print('='*50)
    # mode 0: use the passed "model" to get prediction on "test_fold", not used now
    # mode 1: use the passed "model" to get encoded SNPs and return them, can then be used for other models

    print('begin prediction')
    assert mode in (0, 1)
    y, y_true, all_encode_x = [], [], []
    batch_index = 0

    with torch.no_grad():
        with open('/home/ubuntu/ABCD_genotype/ABCD_QCed_EA_pruned_v2.ped.temp4') as f:
            while True:
                chunk = list(islice(f, batch))
                if not chunk:
                    break
                    
                batch_index += 1
                if batch_index in test_fold:
                    all_data = []
                    all_label = []
                    for line in chunk:
                        orig_key = "".join(line.split('\t')[0].split('_')[1:])
                        if orig_key in combine.index:
                            all_label.append(Y.loc[orig_key, y_columns[0]])
                            all_data.append([int(i) for i in line.split('\t')[6:] if i])
                    if not all_label:
                        del all_data
                        gc.collect()
                        print('Not found data in this batch!')
                        continue

                    img = torch.FloatTensor(all_data)
                    img = img/max_abcd
                    img = img.view(img.size(0), -1)
                    del all_data
                    gc.collect()

                    y_true.extend(all_label)

                    if mode==0:
                        predict_y = model.encode_and_predict(img).cpu()
                        y.extend([ii*(max_label-min_label) + min_label for ii in predict_y.cpu()])
                    else:
                        encode_x = model.encode(img).cpu()
                        all_encode_x.append(torch.cat((encode_x, prs_data), 1))

                    del img
                    gc.collect()

                    #if debug and batch_index > 1: # early return in debug mode
                    #    if mode == 0:
                    #        return sum(((y[i] - y_true[i])**2 for i in range(len(y)))) / len(y)
                    #    else:
                    #        return torch.cat(all_encode_x, 0).numpy(), np.array(y_true)

    
    if mode==0:
        return torch.cat(y, 0).numpy(), np.array(y_true)
    else:
        return torch.cat(all_encode_x, 0).numpy(), np.array(y_true)


In [10]:
reconstruct_criterion = nn.MSELoss()
if binary_output:
    #print('output is binary')
    criterion = nn.CrossEntropyLoss()
else:
    #print('output is continuous')
    criterion = nn.MSELoss()

In [11]:
y_columns = ['nihtbx_totalcomp_uncorrected'] # label dimension is 1
y_dim = len(y_columns)
Y = pd.DataFrame(combine, columns = y_columns)

In [12]:
print('----------------- Use Only SNPs -----------------')
print('batch size:', batch, 'epochs:', num_epochs, 'ae middle dim:', m)
print('classify middles:', classify_middle, 'lr:', lr)

all_folds = []
for i in range(k_fold):
    batches_per_fold = each_fold_size // batch
    all_folds.append(list(range(i*batches_per_fold, (i+1)*batches_per_fold)))

for i in range(len(all_folds)):
    if i==1: #일단 한 번만 test 하기 위함. 나중에 뺄 것.
        break
        
    print('test fold:', all_folds[i])
    train_folds = []
    for j in range(len(all_folds)):
        if j != i:
            train_folds.extend(all_folds[j])

    model = train_nn(lr, m, batch, train_folds)
    torch.save(model.state_dict(), 'NIH_save_epoch100_0730.pth')
    
    print(loss_result)
    
    
    plt.plot(loss_result['epoch'], loss_result['regression loss'])
    plt.plot(loss_result['epoch'], loss_result['reconstuct loss'])
    plt.show()
    
    plt.plot(loss_result['epoch'], loss_result['r squared'])
    plt.show()
    
    if not use_rf:
        y_pred, y_true = make_prediction(list(range(num_samples)), model, batch, 0)
        
        assert len(y_pred) == len(y_true)
        print('total number of samples: ', len(y_pred))
        
        print('Test MSE: ', criterion(torch.tensor(y_pred), torch.tensor(y_true)).item())
        print('R squared score: ', r2_score(y_true, y_pred))
    
    else:
        # prediction using encoded SNPs and prs with RF model
        rf_x, rf_y = make_prediction(list(range(num_samples)), model, batch, 1)
        assert rf_x.shape[0] == rf_y.shape[0]
        print('total number of samples: ', rf_x.shape[0])

        if binary_output:
            rf_model = RandomForestClassifier(n_estimators=100, max_depth=20, random_state=2)
        else:
            rf_model = RandomForestRegressor(n_estimators=100, max_depth=20, random_state=2)

        test_index = list(range(all_folds[i][0]*batch, min((1+all_folds[i][-1])*batch, num_samples)))
        train_index = [i for i in range(num_samples) if i not in test_index]

        rf_model.fit(rf_x[train_index], rf_y[train_index])
        if binary_output:
            print('Train res: ', criterion(torch.tensor(rf_model.predict_proba(rf_x[train_index])), torch.tensor(rf_y[train_index])).item())
            print('Test res: ', criterion(torch.tensor(rf_model.predict_proba(rf_x[test_index])), torch.tensor(rf_y[test_index])).item())
        else:
            y_test_true = rf_y[test_index]
            y_test_prediction = rf_model.predict(rf_x[test_index])
            print('Train res: ', criterion(torch.tensor(rf_model.predict(rf_x[train_index])), torch.tensor(rf_y[train_index])).item())
            print('Test res: ', criterion(torch.tensor(y_test_prediction), torch.tensor(y_test_true)).item())
            print('R squared score: ', r2_score(y_test_true, y_test_prediction))
            
    # prediction using ae regression network, not used now
    #print("train res: ", make_prediction(train_folds, model, batch))
    #print("res: ", make_prediction(all_folds[i], model, batch))
    del model
    gc.collect()

----------------- Use Only SNPs -----------------
batch size: 50 epochs: 100 ae middle dim: 64
classify middles: [16] lr: 0.01
test fold: [0, 1, 2, 3, 4, 5, 6, 7]
--------------------------------------------------
Training........
epoch:  0
count:  1
count:  2
count:  3
count:  4
count:  5
count:  6
count:  7
count:  8
count:  9
count:  10
count:  11
count:  12
count:  13
count:  14
count:  15
count:  16
count:  17
count:  18
count:  19
count:  20
count:  21
count:  22
count:  23
count:  24
count:  25
count:  26
count:  27
count:  28
count:  29
count:  30
count:  31
count:  32
count:  33
count:  34
count:  35
count:  36
count:  37
count:  38
count:  39
count:  40
count:  41
count:  42
count:  43
count:  44
count:  45
count:  46
count:  47
count:  48
count:  49
count:  50
count:  51
count:  52
count:  53
count:  54
count:  55
count:  56
count:  57
count:  58
count:  59
count:  60
count:  61
count:  62
count:  63
count:  64
count:  65
count:  66
count:  67
count:  68
count:  69
count:  7

KeyboardInterrupt: 

In [13]:
loss_result

,epoch,reconstuct loss,regression loss,r squared
0,0.0,0.038488,0.146949,-16.408255
1,1.0,0.038498,0.016513,-0.956165
